In [1]:
%matplotlib widget
import os
import sys
import re
import numpy as np
import matplotlib
import serial
import pyvisa
import csv
import struct as struct
from time import sleep, time
from pylab import *
from datetime import datetime

from scipy import integrate
from scipy.special import jv

import pylab as pyplt
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
from scipy.interpolate import interp2d

import codecs
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm

In [2]:
def readAmpCSVSwept(filePath):
    angle = []
    amp = []
    max = -100
    with open(filePath, newline='') as csvfile:
        file = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in file:
            if float(row[1]) > max:
                max = float(row[1])
        
    with open(filePath, newline='') as csvfile:
        file = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in file:
            amp.append(float(row[1]) - max)
            
            
    return amp, max

def writeAUTData2File_3D(filepath, filenamenotype, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if(pol == 0): #p-polarized
                    azimuthDeg_p_pol.append(float(line[0:9]))
                    elevationDeg_p_pol.append(float(line[10:19]))
                    ampdb_p_pol.append(float(line[20:29]))
                    phaseDeg_p_pol.append(float(line[30:]))
                elif(pol > 0): #x-polarized
                    azimuthDeg_x_pol.append(float(line[0:9]))
                    elevationDeg_x_pol.append(float(line[10:19]))
                    ampdb_x_pol.append(float(line[20:29]))
                    phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                pol += 1
                next(f)
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)
    
    np.savetxt(filepath + filenamenotype + "/azimuthDeg_p_pol.csv", azimuthDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/elevationDeg_p_pol.csv", elevationDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/ampdb_p_pol.csv", ampdb_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/phaseDeg_p_pol.csv", phaseDeg_p_pol, delimiter=",")
    
    np.savetxt(filepath + filenamenotype + "/azimuthDeg_x_pol.csv", azimuthDeg_x_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/elevationDeg_x_pol.csv", elevationDeg_x_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/ampdb_x_pol.csv", ampdb_x_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/phaseDeg_x_pol.csv", phaseDeg_x_pol, delimiter=",")
            
def writeAUTData2File_2D(filepath, filenamenotype, elevation, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if (elevation == float(line[10:19])):
                    if(pol == 0): #p-polarized
                        azimuthDeg_p_pol.append(float(line[0:9]))
                        elevationDeg_p_pol.append(float(line[10:19]))
                        ampdb_p_pol.append(float(line[20:29]))
                        phaseDeg_p_pol.append(float(line[30:]))
                    elif(pol > 0): #x-polarized
                        azimuthDeg_x_pol.append(float(line[0:9]))
                        elevationDeg_x_pol.append(float(line[10:19]))
                        ampdb_x_pol.append(float(line[20:29]))
                        phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                next(f)
                pol += 1
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)

    np.savetxt(filepath + filenamenotype + "/azimuthDeg_p_pol_" + str(elevation) + "_elevation.csv", azimuthDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/elevationDeg_p_pol_" + str(elevation) + "_elevation.csv", elevationDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/ampdb_p_pol_" + str(elevation) + "_elevation.csv", ampdb_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/phaseDeg_p_pol_" + str(elevation) + "_elevation.csv", phaseDeg_p_pol, delimiter=",")
    
    np.savetxt(filepath + filenamenotype + "/azimuthDeg_x_pol_" + str(elevation) + "_elevation.csv", azimuthDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/elevationDeg_x_pol_" + str(elevation) + "_elevation.csv", elevationDeg_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/ampdb_x_pol_" + str(elevation) + "_elevation.csv", ampdb_p_pol, delimiter=",")
    np.savetxt(filepath + filenamenotype + "/phaseDeg_x_pol_" + str(elevation) + "_elevation.csv", phaseDeg_p_pol, delimiter=",")
            
        
def returnAUTData_3D(filepath, filenamenotype, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if(pol == 0): #p-polarized
                    azimuthDeg_p_pol.append(float(line[0:9]))
                    elevationDeg_p_pol.append(float(line[10:19]))
                    ampdb_p_pol.append(float(line[20:29]))
                    phaseDeg_p_pol.append(float(line[30:]))
                elif(pol > 0): #x-polarized
                    azimuthDeg_x_pol.append(float(line[0:9]))
                    elevationDeg_x_pol.append(float(line[10:19]))
                    ampdb_x_pol.append(float(line[20:29]))
                    phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                pol += 1
                next(f)
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)
    
    return azimuthDeg_p_pol, elevationDeg_p_pol, ampdb_p_pol, azimuthDeg_x_pol, elevationDeg_x_pol, ampdb_x_pol
            
def returnAUTData_2D(filepath, filenamenotype, elevation, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if (elevation == float(line[10:19])):
                    if(pol == 0): #p-polarized
                        azimuthDeg_p_pol.append(float(line[0:9]))
                        elevationDeg_p_pol.append(float(line[10:19]))
                        ampdb_p_pol.append(float(line[20:29]))
                        phaseDeg_p_pol.append(float(line[30:]))
                    elif(pol > 0): #x-polarized
                        azimuthDeg_x_pol.append(float(line[0:9]))
                        elevationDeg_x_pol.append(float(line[10:19]))
                        ampdb_x_pol.append(float(line[20:29]))
                        phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                next(f)
                pol += 1
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)

    return azimuthDeg_p_pol, elevationDeg_p_pol, ampdb_p_pol, azimuthDeg_x_pol, elevationDeg_x_pol, ampdb_x_pol
            
def returnAUTData_2D_septum_reverse(filepath, filenamenotype, azimuth, linestart):
    from pathlib import Path


    #data arrays
    azimuthDeg_p_pol = []
    elevationDeg_p_pol = []
    ampdb_p_pol = []
    phaseDeg_p_pol = []
    azimuthDeg_x_pol = []
    elevationDeg_x_pol = []
    ampdb_x_pol = []
    phaseDeg_x_pol = []

    #variables from file
    azimuthSpan = 360
    azimuthStart = -180
    azimuthStop = 180
    azimuthPts = 201
    azimuthDelta = 1.8
    elevationSpan = 20
    elevationStart = -10
    elevationStop = 10
    elevationPts = 201
    elevationDelta = 0.1

    with open(filepath + filenamenotype + ".txt") as f:
        for _ in range(linestart):
            next(f)

        pol = 0
        for line in f:
            try:
                if (azimuth == float(line[0:9])):
                    if(pol == 0): #p-polarized
                        azimuthDeg_p_pol.append(float(line[0:9]))
                        elevationDeg_p_pol.append(float(line[10:19]))
                        ampdb_p_pol.append(float(line[20:29]))
                        phaseDeg_p_pol.append(float(line[30:]))
                    elif(pol > 0): #x-polarized
                        azimuthDeg_x_pol.append(float(line[0:9]))
                        elevationDeg_x_pol.append(float(line[10:19]))
                        ampdb_x_pol.append(float(line[20:29]))
                        phaseDeg_x_pol.append(float(line[30:]))
            except ValueError:
                next(f)
                pol += 1
                
    Path(filepath + filenamenotype).mkdir(parents=True, exist_ok=True)

    return azimuthDeg_p_pol, elevationDeg_p_pol, ampdb_p_pol, azimuthDeg_x_pol, elevationDeg_x_pol, ampdb_x_pol

In [7]:
#writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15")
#writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15")
for i in arange(1,31,1):
    try:
        writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat_new/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam"+str(i), 79)
    except:
        break
        
for i in arange(1,31,1):
    try:
        writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat_new/Hermes3U-horizontal/","Hermes2U-20211004_beam"+str(i), 79)
    except:
        break
        
for i in arange(1,31,1):
    try:
        writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat_new/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004_beam"+str(i), 79)
    except:
        break
        
for i in arange(1,31,1):
    try:
        writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat_new/Hermes2U-vertical/","Hermes2U-Vertical-20211004_beam"+str(i), 79)
    except:
        break
        
for i in arange(1,31,1):
    try:
        writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat_new/Polarizer/","SEPTUM_POLARIZER-20211004_beam"+str(i), 81)
    except:
        break

In [10]:

writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat_new/Polarizer/","SEPTUM_POLARIZER_beam15", 81)


In [282]:
#writeAUTData2File_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 0)
#writeAUTData2File_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 0)
writeAUTData2File_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 81)

In [3]:
##############################Initialise#####################################################
nrow = 2
ncol = 3
xsize = 11.5
ysize = 8
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 2U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 0, 79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 5, 79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 10, 79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')

p_pol_0 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_0 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = 'black',label=r'0$\degree$')

x_pol_0 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_0 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')

#hermes 2U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 0, 79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", -5, 79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", -10, 79)

#plots
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')

x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')

#limits
axes[0,0].set_ylim(-40,0)
axes[0,0].set_yticks(np.arange(-40,0,10))
axes[0,1].set_ylim(-40,0)
axes[0,1].set_yticks(np.arange(-30,0,10))
axes[0,2].set_ylim(-20,0)
axes[0,2].set_yticks(np.arange(-20,0,5))

axes[1,0].set_ylim(-40,0)
axes[1,0].set_yticks(np.arange(-40,0,10))
axes[1,1].set_ylim(-40,0)
axes[1,1].set_yticks(np.arange(-30,0,10))
axes[1,2].set_ylim(-20,0)
axes[1,2].set_yticks(np.arange(-20,0,5))

#legend
axes[0,2].legend()

#titles
#plt.rcParams['axes.titlepad'] = 10

fig.suptitle(r'Hermes 2U - Horizontal Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
##############################Initialise#####################################################
nrow = 2
ncol = 3
xsize = 11.5
ysize = 8
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 2U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 0, 79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 5, 79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 10, 79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')

p_pol_0 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_0 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = 'black',label=r'0$\degree$')

x_pol_0 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_0 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')

#hermes 2U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", 0, 79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", -5, 79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", -10, 79)

#plots
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')

x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')

#limits
axes[0,0].set_ylim(-40,0)
axes[0,0].set_yticks(np.arange(-40,0,10))
axes[0,1].set_ylim(-40,0)
axes[0,1].set_yticks(np.arange(-30,0,10))
axes[0,2].set_ylim(-20,0)
axes[0,2].set_yticks(np.arange(-20,0,5))

axes[1,0].set_yticks(np.arange(-40,0,10))
axes[1,1].set_ylim(-40,0)
axes[1,1].set_yticks(np.arange(-30,0,10))
axes[1,2].set_ylim(-20,0)
axes[1,2].set_yticks(np.arange(-20,0,5))

#legend
axes[0,2].legend()

#titles
#plt.rcParams['axes.titlepad'] = 10

fig.suptitle(r'Hermes 2U - Vertical Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
##############################Initialise#####################################################
nrow = 2
ncol = 3
xsize = 11.5
ysize = 8
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 0,79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 5,79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 10,79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')

p_pol_0 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_0 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = 'black',label=r'0$\degree$')

x_pol_0 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_0 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')

#hermes 3U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 0,79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", -5,79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", -10,79)

#plots
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')

x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')

#limits
axes[0,0].set_ylim(-10,0)
axes[0,0].set_yticks(np.arange(-40,0,10))
axes[0,1].set_ylim(-40,0)
axes[0,1].set_yticks(np.arange(-30,0,10))
axes[0,2].set_ylim(-20,0)
axes[0,2].set_yticks(np.arange(-20,0,5))

axes[1,0].set_yticks(np.arange(-40,0,10))
axes[1,1].set_ylim(-40,0)
axes[1,1].set_yticks(np.arange(-30,0,10))
axes[1,2].set_ylim(-20,0)
axes[1,2].set_yticks(np.arange(-20,0,5))

#legend
axes[0,2].legend()

#titles

fig.suptitle(r'Hermes 3U - Horizontal Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
##############################Initialise#####################################################
nrow = 1
ncol = 1
xsize = 5
ysize = 5
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", 0,79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes.plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')
axes.plot([315*3.1415/180,315*3.1415/180],[-5,0] , 'r--')
axes.plot([27*3.1415/180,27*3.1415/180],[-5,0] , 'r--')

#limits
axes.set_ylim(-4,0)
axes.set_yticks(np.arange(-5,0,1))
#legend

#titles

fig.suptitle(r'Hermes 3U - Horizontal Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes.set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes.set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
##############################Initialise#####################################################
nrow = 1
ncol = 1
xsize = 5
ysize = 5
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", 0,79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes.plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')
big = 319
small = 33
axes.plot([big*3.1415/180,big*3.1415/180],[-5,0] , 'r--')
axes.plot([small*3.1415/180,small*3.1415/180],[-5,0] , 'r--')

#limits
axes.set_ylim(-4,0)
axes.set_yticks(np.arange(-5,0,1))
#legend

#titles

fig.suptitle(r'Hermes 2U - Horizontal Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes.set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes.set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
##############################Initialise#####################################################
nrow = 2
ncol = 3
xsize = 11.5
ysize = 8
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", 0,79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", 5,79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", 10,79)

#plots
cmap = cm.get_cmap('viridis', 3)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = 'black',label=r'0$\degree$')

p_pol_0 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_0 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = 'black',label=r'0$\degree$')

x_pol_0 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_0 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')

#hermes 3U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", 0,79)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", -5,79)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", -10,79)

#plots
p_pol_5 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_5 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,2].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')

x_pol_5 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_5 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,2].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')

#limits
axes[0,0].set_ylim(-10,0)
axes[0,0].set_yticks(np.arange(-40,0,10))
axes[0,1].set_ylim(-40,0)
axes[0,1].set_yticks(np.arange(-30,0,10))
axes[0,2].set_ylim(-20,0)
axes[0,2].set_yticks(np.arange(-20,0,5))

axes[1,0].set_yticks(np.arange(-40,0,10))
axes[1,1].set_ylim(-40,0)
axes[1,1].set_yticks(np.arange(-30,0,10))
axes[1,2].set_ylim(-20,0)
axes[1,2].set_yticks(np.arange(-20,0,5))

#legend
axes[0,2].legend()

#titles

fig.suptitle(r'Hermes 3U - Vertical Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
##############################Initialise#####################################################
nrow = 2
ncol = 2
xsize = 8
ysize = 8.5
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))

fig.subplots_adjust(hspace=0.025, wspace=0.025)

fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 0.385,81)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 5,81)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 10.385,81)
azimuthDeg_p_pol_15, elevationDeg_p_pol_15, ampdb_p_pol_15, azimuthDeg_x_pol_15, elevationDeg_x_pol_15, ampdb_x_pol_15 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 15,81)

#plots
cmap = cm.get_cmap('viridis', 4)

p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1))
p_pol_10 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2))
p_pol_15 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_15,3.1415/180), ampdb_p_pol_15, color = cmap(3))

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1))
x_pol_10 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2))
x_pol_15 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_15,3.1415/180), ampdb_x_pol_15, color = cmap(3))

#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 0.385,81)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 5,81)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 10.385,81)
azimuthDeg_p_pol_15, elevationDeg_p_pol_15, ampdb_p_pol_15, azimuthDeg_x_pol_15, elevationDeg_x_pol_15, ampdb_x_pol_15 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 15,81)

p_pol_0 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'0$^\circ$')
p_pol_5 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'10$^\circ$')
p_pol_15 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_15,3.1415/180), ampdb_p_pol_15, color = cmap(2),label=r'15$^\circ$')


x_pol_0 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'0$^\circ$')
x_pol_5 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'10$^\circ$')
x_pol_15 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_15,3.1415/180), ampdb_x_pol_15, color = cmap(2),label=r'15$^\circ$')

#hermes 3U negative
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", -0.385,81)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", -5,81)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", -10.385,81)
azimuthDeg_p_pol_15, elevationDeg_p_pol_15, ampdb_p_pol_15, azimuthDeg_x_pol_15, elevationDeg_x_pol_15, ampdb_x_pol_15 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", -15,81)

#plots
p_pol_0 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0))
p_pol_5 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_15 = axes[0,0].plot(np.multiply(azimuthDeg_p_pol_15,3.1415/180), ampdb_p_pol_15, color = cmap(3),label=r'-10$^\circ$')

x_pol_0 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_0,3.1415/180), ampdb_x_pol_0, color = cmap(0))
x_pol_5 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_15 = axes[1,0].plot(np.multiply(azimuthDeg_x_pol_15,3.1415/180), ampdb_x_pol_15, color = cmap(3),label=r'-10$^\circ$')

#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 0.385,81)
azimuthDeg_p_pol_5, elevationDeg_p_pol_5, ampdb_p_pol_5, azimuthDeg_x_pol_5, elevationDeg_x_pol_5, ampdb_x_pol_5 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 5,81)
azimuthDeg_p_pol_10, elevationDeg_p_pol_10, ampdb_p_pol_10, azimuthDeg_x_pol_10, elevationDeg_x_pol_10, ampdb_x_pol_10 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 10.385,81)
azimuthDeg_p_pol_15, elevationDeg_p_pol_15, ampdb_p_pol_15, azimuthDeg_x_pol_15, elevationDeg_x_pol_15, ampdb_x_pol_15 = returnAUTData_2D_septum_reverse("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", 15,81)

#plots
p_pol_0 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_0,3.1415/180), ampdb_p_pol_0, color = cmap(0),label=r'-0$^\circ$')
p_pol_5 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_5,3.1415/180), ampdb_p_pol_5, color = cmap(1),label=r'-5$^\circ$')
p_pol_10 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_10,3.1415/180), ampdb_p_pol_10, color = cmap(2),label=r'-10$^\circ$')
p_pol_15 = axes[0,1].plot(np.multiply(elevationDeg_p_pol_15,3.1415/180), ampdb_p_pol_15, color = cmap(3),label=r'-15$^\circ$')

x_pol_0 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_5,3.1415/180), ampdb_x_pol_0, color = cmap(0),label=r'-0$^\circ$')
x_pol_5 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_5,3.1415/180), ampdb_x_pol_5, color = cmap(1),label=r'-5$^\circ$')
x_pol_10 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_10,3.1415/180), ampdb_x_pol_10, color = cmap(2),label=r'-10$^\circ$')
x_pol_15 = axes[1,1].plot(np.multiply(elevationDeg_x_pol_15,3.1415/180), ampdb_x_pol_15, color = cmap(3),label=r'-15$^\circ$')

#limits
axes[0,0].set_ylim(-5,0)
axes[0,0].set_yticks(np.arange(-5,0,1))
axes[0,1].set_ylim(-5,0)
axes[0,1].set_yticks(np.arange(-5,0,1))

axes[1,1].set_ylim(-5,0)
axes[1,0].set_yticks(np.arange(-5,0,1))
axes[1,1].set_ylim(-5,0)
axes[1,1].set_yticks(np.arange(-5,0,1))

#legend
axes[0,1].legend()

#titles

fig.suptitle(r'Septum Polarizer - Plane Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0,0].set_ylabel(r'P-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_ylabel(r'X-Polarized', fontsize = labelfont,labelpad=30)
axes[1,0].set_xlabel(r'Azimuth', fontsize = labelfont,labelpad=10)
axes[1,1].set_xlabel(r'Elevation', fontsize = labelfont,labelpad=10)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
##############################Initialise#####################################################
nrow = 1
ncol = 3
xsize = 9
ysize = 4
plt.close('all')

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize),subplot_kw=dict(projection='polar'))
fig.subplots_adjust(hspace=0.025, wspace=0.025)
fig.set_facecolor('w')


matplotlib.pyplot.tight_layout()

#hermes 3U positive
#data
amp0, max0 = readAmpCSVSwept("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/rad_testing_nov26/CP_2021-11-26 17h32m42s -  Nadir Up-theta0-phiswept - Copy.csv")
amp1, max1 = readAmpCSVSwept("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/rad_testing_nov26/CP_2021-11-26 17h19m38s - Starbaord Up-theta90phiswept - Copy.csv")
amp2, max2 = readAmpCSVSwept("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/rad_testing_nov26/UHF-Arke_Radiation_Testing_Results_20211126.csv")



pi = 3.14159
angle0 = np.linspace(-pi/4,2*pi-pi/4,len(amp0))
angle1 = np.linspace(0,2*pi,len(amp1))
angle2 = np.linspace(0,2*pi,len(amp2))

#plots
cmap = cm.get_cmap('viridis', 3)


plot0 = axes[0].plot(angle0, amp0, color = 'black',label=r'$\theta$ = 0$^\circ$')
plot1 = axes[1].plot(angle1, amp1, color = 'black',label=r'$\theta$ = 90$^\circ$')
plot2 = axes[2].plot(angle2, amp2, color = 'black',label=r'$\phi$ = 0$^\circ$')


#limits
axes[0].set_ylim(-10,0)
axes[0].set_yticks(np.arange(-10,0,2.5))
axes[1].set_ylim(-10,0)
axes[1].set_yticks(np.arange(-10,0,2.5))
axes[2].set_ylim(-10,0)
axes[2].set_yticks(np.arange(-10,0,2.5))
#legend
axes[0].set_xlabel(r'$\phi$ = 0$^\circ$, ref ='+str(max0)+'dBm', fontsize = labelfont,labelpad=10)
axes[1].set_xlabel(r'$\phi$ = 90$^\circ$, ref ='+str(max1)+'dBm', fontsize = labelfont,labelpad=10)
axes[2].set_xlabel(r'$\theta$ = 0$^\circ$, ref ='+str(max2)+'dBm', fontsize = labelfont,labelpad=10)

#titles
fig.suptitle(r'Arke 3U 437.875 MHz', fontsize = titlefont, y = 1)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
##############################Initialise#####################################################
nrow = 1
ncol = 2
xsize = 8
ysize = 3.7
plt.close('all')
pi = 3.14159

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 12 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))

fig.subplots_adjust(hspace=0.025, wspace=0.005)

fig.set_facecolor('w')

#x_locator = FixedLocator([lambda_s_center-lambda_s_width/4, lambda_s_center, lambda_s_center+lambda_s_width/4])
#x_formatter = FixedFormatter([0,r'$\pi/4$',r'$\pi/2$'])
#y_locator = FixedLocator([lambda_i_center-lambda_i_width/4, lambda_i_center, lambda_i_center+lambda_i_width/4])
#y_formatter = FixedFormatter([791.6,792.1,792.6])
x_locator = FixedLocator([-15, -10, -5, 0, 5, 10, 15])
#y_locator = FixedLocator([791.5, 792, 792.5])

for i in range(0,len(axes)):
    axes[i].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    axes[i].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    axes[i].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
    axes[i].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
#    axes[i].xaxis.set_major_formatter(x_formatter)
    axes[i].xaxis.set_major_locator(x_locator)
    #axes[i].yaxis.set_major_formatter(y_formatter)
    axes[i].yaxis.set_major_locator(x_locator)
#    axes[i].set_xlim(-15,105)
#    axes[i].set_ylim(-0.1,2.6)
    
#axes[1].set_yticklabels([])

#labels

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15",81)
ampdb = zeros((40,40))
ampdb_x = zeros((40,40))
azimuth = zeros(40)
for i in range(40):
    azimuth[i] = (-15+15/20*i)
    for j in range(40):
        theta = (-15+15/20*j)*pi/180
        phi = (-15 + 15/20*i)*pi/180
        #scale = 10*log10((np.cos(theta)**2)*(np.cos(phi)**2))
        #print(scale)
        #ampdb[39-i,j] = 10*log10(scale)# + ampdb_p_pol_0[j+40*i]
        ampdb[39-i,j] = ampdb_p_pol_0[j+40*i]# + scale
        ampdb_x[39-i,j] = ampdb_x_pol_0[j+40*i]# + scale
        
        
#plots
#axes[0].pcolormesh(azimuthDeg_p_pol_0[0:40], azimuthDeg_p_pol_0[0:40], ampdb, cmap='viridis')
p1 = axes[0].pcolormesh(azimuth, azimuth, ampdb, cmap='viridis',vmin=-6, vmax=0)
p2 = axes[1].pcolormesh(azimuth, azimuth, ampdb_x, cmap='viridis',vmin=-6, vmax=0)
fig.colorbar(p1, ax=axes[0])
fig.colorbar(p2, ax=axes[1])
#limits
#axes[0,0].set_ylim(-5,0)
#axes[0,0].set_yticks(np.arange(-5,0,1))
#axes[0,1].set_ylim(-5,0)
#axes[0,1].set_yticks(np.arange(-5,0,1))

#axes[1,1].set_ylim(-5,0)
#axes[1,0].set_yticks(np.arange(-5,0,1))
#axes[1,1].set_ylim(-5,0)
#axes[1,1].set_yticks(np.arange(-5,0,1))

#legend
#axes[0,1].legend()

#titles

#fig.suptitle(r'Septum Polarizer - Plane Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0].set_ylabel(r'Elevation ($^\circ$)', fontsize = labelfont,labelpad=5)
axes[0].set_xlabel(r'Azimuth - P Polarization ($^\circ$)', fontsize = labelfont,labelpad=5)
axes[1].set_xlabel(r'Azimuth - X Polarization ($^\circ$)', fontsize = labelfont,labelpad=5)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
##############################Initialise#####################################################
nrow = 1
ncol = 2
xsize = 8
ysize = 4.3
plt.close('all')
pi = 3.14159

#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 10.0 #major tick length
minw = axew #minor tick width
minl = 5.0 #minor tick length

#set global font sizes
axefont = 15 #axis label font size
numsize = 15 #axis number font size
legfont = 15 #legend font size
labelfont=15
titlefont = 20


fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))

fig.subplots_adjust(hspace=0.025, wspace=0.005)

fig.set_facecolor('w')

#x_locator = FixedLocator([lambda_s_center-lambda_s_width/4, lambda_s_center, lambda_s_center+lambda_s_width/4])
#x_formatter = FixedFormatter([0,r'$\pi/4$',r'$\pi/2$'])
#y_locator = FixedLocator([lambda_i_center-lambda_i_width/4, lambda_i_center, lambda_i_center+lambda_i_width/4])
#y_formatter = FixedFormatter([791.6,792.1,792.6])
#x_locator = FixedLocator([-15, -10, -5, 0, 5, 10, 15])
#y_locator = FixedLocator([791.5, 792, 792.5])

for i in range(0,len(axes)):
    axes[i].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    axes[i].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    axes[i].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
    axes[i].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
#    axes[i].xaxis.set_major_formatter(x_formatter)
#    axes[i].xaxis.set_major_locator(x_locator)
    #axes[i].yaxis.set_major_formatter(y_formatter)
#    axes[i].yaxis.set_major_locator(x_locator)
#    axes[i].set_xlim(-15,105)
#    axes[i].set_ylim(-0.1,2.6)
    


#labels

#hermes 3U positive
#data
azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_3D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15",81)
ampdb = zeros((40,40))
ampdb_x = zeros((40,40))
azimuth = zeros(40)
for i in range(40):
    azimuth[i] = (-15+15/20*i)
    for j in range(40):
        theta = (-15+15/20*j)*pi/180
        phi = (-15 + 15/20*i)*pi/180
        #scale = 10*log10((np.cos(theta)**2)*(np.cos(phi)**2))
        #print(scale)
        #ampdb[39-i,j] = 10*log10(scale)# + ampdb_p_pol_0[j+40*i]
        ampdb[39-i,j] = ampdb_p_pol_0[j+40*i]# + scale
        ampdb_x[39-i,j] = ampdb_x_pol_0[j+40*i]# + scale
        
        
#plots
axes[0].plot([-15,15],[-3,-3],'--k')
axes[1].plot([-15,15],[-3,-3],'--k')
p_pol_0 = axes[0].plot(azimuth, ampdb[19], color = 'red')
x_pol_0 = axes[1].plot(azimuth, ampdb_x[19], color = 'red')
axes[0].plot([-15.1,-15.1],[-5,0],'--k')
axes[1].plot([-14.94,-14.94],[-5,0],'--k')
axes[0].plot([12.54,12.54],[-5,0],'--k')
axes[1].plot([13.9,13.9],[-5,0],'--k')
#limits
print(str(12.54+15.1))
print(str(14.94+13.9))
#axes[0,0].set_ylim(-5,0)
#axes[0,0].set_yticks(np.arange(-5,0,1))
#axes[0,1].set_ylim(-5,0)
#axes[0,1].set_yticks(np.arange(-5,0,1))

axes[1].set_ylim(-4,0)
axes[0].set_ylim(-4,0)
axes[1].set_xlim(-17,17)
axes[0].set_xlim(-17,17)
#axes[1,0].set_yticks(np.arange(-5,0,1))
#axes[1,1].set_ylim(-5,0)
#axes[1,1].set_yticks(np.arange(-5,0,1))

#legend
#axes[0,1].legend()

#titles

#fig.suptitle(r'Septum Polarizer - Plane Measurement - 2.428 GHz', fontsize = titlefont, y = 1)
axes[0].set_ylabel(r'Amplitude (dB)', fontsize = labelfont,labelpad=5)
axes[1].set_ylabel(r'Amplitude (dB)', fontsize = labelfont,labelpad=5)
axes[0].set_xlabel(r'Azimuth - P Polarization ($^\circ$)', fontsize = labelfont,labelpad=5)
axes[1].set_xlabel(r'Azimuth - X Polarization ($^\circ$)', fontsize = labelfont,labelpad=5)
plt.tight_layout()
#fig = plt.figure()
#ax = plt.axes(projection='3d')

#ax.plot_surface(lambda_s_m, lambda_i_m, JSI**2,cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

27.64
28.84


In [12]:
max(ampdb_x[19])

-0.169

In [61]:
#integrating
result_p = zeros(len(arange(-10,10,0.1)))
result_x = zeros(len(arange(-10,10,0.1)))
#fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(5,5))
elevation = arange(-100,100,1)
for scannedLine in range(len(elevation)):
    try:
        azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-horizontal/","Hermes2U-Horizontal-20211004-Actual_beam15", elevation[scannedLine]/10,79)
        
        amp_p = zeros(len(ampdb_p_pol_0))
        amp_x = zeros(len(ampdb_x_pol_0))
        for i in range(len(ampdb_p_pol_0)):
            amp_p[i] = 10**(ampdb_p_pol_0[i]/10)
            amp_x[i] = 10**(ampdb_x_pol_0[i]/10)
        result_p[scannedLine] =np.trapz(amp_p, azimuthDeg_p_pol_0)/360/max(amp_p)
        result_x[scannedLine] =np.trapz(amp_x, azimuthDeg_x_pol_0)/360/max(amp_x)

        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_p, label = elevation[scannedLine]/10)
        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_x, label = elevation[scannedLine]/10)
        #axes[scannedLine].legend()
    except:
       # print(scannedLine)
       # print(elevation[scannedLine])
        break
print(10*log10(np.mean(np.add(result_p,result_x))))

-2.316958741232052


In [60]:
#integrating
result_p = zeros(len(arange(-10,10,0.1)))
result_x = zeros(len(arange(-10,10,0.1)))
#fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(5,5))
elevation = arange(-100,100,1)
for scannedLine in range(len(elevation)):
    try:
        azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes2U-vertical/","Hermes2U-Vertical-20211004-Actual_beam15", elevation[scannedLine]/10,79)
        
        amp_p = zeros(len(ampdb_p_pol_0))
        amp_x = zeros(len(ampdb_x_pol_0))
        for i in range(len(ampdb_p_pol_0)):
            amp_p[i] = 10**(ampdb_p_pol_0[i]/10)
            amp_x[i] = 10**(ampdb_x_pol_0[i]/10)
        result_p[scannedLine] =np.trapz(amp_p, azimuthDeg_p_pol_0)/360/max(amp_p)
        result_x[scannedLine] =np.trapz(amp_x, azimuthDeg_x_pol_0)/360/max(amp_x)

        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_p, label = elevation[scannedLine]/10)
        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_x, label = elevation[scannedLine]/10)
        #axes[scannedLine].legend()
    except:
       # print(scannedLine)
       # print(elevation[scannedLine])
        break
10*log10(np.mean(np.add(result_p,result_x)))

-2.548419480587456

In [63]:
#integrating
result_p = zeros(len(arange(-10,10,0.1)))
result_x = zeros(len(arange(-10,10,0.1)))
#fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(5,5))
elevation = arange(-100,100,1)
for scannedLine in range(len(elevation)):
    try:
        azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-horizontal/","Hermes2U-20211004_beam17", elevation[scannedLine]/10,79)
        
        amp_p = zeros(len(ampdb_p_pol_0))
        amp_x = zeros(len(ampdb_x_pol_0))
        for i in range(len(ampdb_p_pol_0)):
            amp_p[i] = 10**(ampdb_p_pol_0[i]/10)
            amp_x[i] = 10**(ampdb_x_pol_0[i]/10)
        result_p[scannedLine] =np.trapz(amp_p, azimuthDeg_p_pol_0)/360/max(amp_p)
        result_x[scannedLine] =np.trapz(amp_x, azimuthDeg_x_pol_0)/360/max(amp_x)

        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_p, label = elevation[scannedLine]/10)
        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_x, label = elevation[scannedLine]/10)
        #axes[scannedLine].legend()
    except:
       # print(scannedLine)
       # print(elevation[scannedLine])
        break
print(10*log10(np.mean(np.add(result_p,result_x))))

-3.0802971571343534


In [120]:
#integrating
result_p = zeros(len(arange(-10,10,0.1)))
result_x = zeros(len(arange(-10,10,0.1)))
#fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(5,5))
elevation = arange(-100,100,1)
for scannedLine in range(len(elevation)):
    try:
        azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Hermes3U-vertical/","Hermes2U-Vertical-20211004_beam17", elevation[scannedLine]/10,79)
        
        amp_p = zeros(len(ampdb_p_pol_0))
        amp_x = zeros(len(ampdb_x_pol_0))
        for i in range(len(ampdb_p_pol_0)):
            amp_p[i] = 10**(ampdb_p_pol_0[i]/10)
            amp_x[i] = 10**(ampdb_x_pol_0[i]/10)
        result_p[scannedLine] =np.trapz(amp_p, azimuthDeg_p_pol_0)/360/max(amp_p)
        result_x[scannedLine] =np.trapz(amp_x, azimuthDeg_x_pol_0)/360/max(amp_x)

        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_p, label = elevation[scannedLine]/10)
        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_x, label = elevation[scannedLine]/10)
        #axes[scannedLine].legend()
    except:
       # print(scannedLine)
       # print(elevation[scannedLine])
        break
print(10*log10(np.mean(np.add(result_p,result_x))))

-2.5572563516223794


In [121]:
print(10*log10(np.mean(np.add(result_p[11],result_x[11]))))

-2.856044063022539


In [119]:
#integrating
result_p = zeros(len(arange(-15,15,0.769)))
result_x = zeros(len(arange(-15,15,0.769)))
#fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(5,5))
elevation = [-15,-14.231,-13.462,-12.692,-11.923,-11.154,-10.385,-9.615,-8.846,-8.077,-7.308,-6.538,-5.769,-5.000,-4.231,-3.462,-2.692,-1.923,-1.154,-0.385,0.385,1.154,1.923,2.692,3.462,4.231,5,5.769,6.538,7.308,8.077,8.846,9.615,10.385,11.154,11.923,12.692,13.462,14.231,15 ]
for scannedLine in range(len(elevation)):
    try:
        azimuthDeg_p_pol_0, elevationDeg_p_pol_0, ampdb_p_pol_0, azimuthDeg_x_pol_0, elevationDeg_x_pol_0, ampdb_x_pol_0 = returnAUTData_2D("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/S-band/Anechoic/AlbertaSat/Polarizer/","SEPTUM_POLARIZER_beam15", elevation[scannedLine],81)
        
        amp_p = zeros(len(ampdb_p_pol_0))
        amp_x = zeros(len(ampdb_x_pol_0))
        for i in range(len(ampdb_p_pol_0)):
            amp_p[i] = 10**(ampdb_p_pol_0[i]/10)
            amp_x[i] = 10**(ampdb_x_pol_0[i]/10)
        result_p[scannedLine] =np.trapz(amp_p, azimuthDeg_p_pol_0)/360/max(amp_p)
        result_x[scannedLine] =np.trapz(amp_x, azimuthDeg_x_pol_0)/360/max(amp_x)

        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_p, label = elevation[scannedLine]/10)
        #axes[scannedLine].plot(azimuthDeg_p_pol_0,amp_x, label = elevation[scannedLine]/10)
        #axes[scannedLine].legend()
    except:
       # print(scannedLine)
       # print(elevation[scannedLine])
        break
print(10*log10(np.mean(np.add(result_p,result_x))))

-8.363012545583947


In [111]:
arange(-15,15,0.769)

array([-15.   , -14.231, -13.462, -12.693, -11.924, -11.155, -10.386,
        -9.617,  -8.848,  -8.079,  -7.31 ,  -6.541,  -5.772,  -5.003,
        -4.234,  -3.465,  -2.696,  -1.927,  -1.158,  -0.389,   0.38 ,
         1.149,   1.918,   2.687,   3.456,   4.225,   4.994,   5.763,
         6.532,   7.301,   8.07 ,   8.839,   9.608,  10.377,  11.146,
        11.915,  12.684,  13.453,  14.222,  14.991])